In [1]:
import modin.pandas as pd
from occupationcoder.coder import SOCCoder
from langdetect import detect, DetectorFactory
from datetime import datetime
import os
import ast

import nltk
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eshad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\eshad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [67]:
directory_paths = ["C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\1. Consolidated data\\2018\\data_2018 backup 29 march\\"]
save_path = "C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\2. Filtered preprocessed data\\2018\\2018_backup_29_mar.csv"

In [68]:
logs = ""

In [69]:
columns = ['jobPostingId', 'companyDetails', 'jobState', 'description', 'title', 'workRemoteAllowed', 'formattedLocation', 'workplaceTypesResolutionResults']
# Initialize an empty list to hold DataFrames
dataframes = []
# Read each CSV file and append to the list
for directory_path in directory_paths:
    csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]
    for csv_file in csv_files:
        # if csv_file.endswith('7.csv') or csv_file.endswith('8.csv') or csv_file.endswith('9.csv') or csv_file.endswith('10.csv'): # or csv_file.endswith('4.csv') or csv_file.endswith('5.csv'):
            file_path = os.path.join(directory_path, csv_file)
            tmp_df = pd.read_csv(file_path, encoding='latin')
            dataframes.append(tmp_df)
            print(f"read {file_path}")

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)
print(len(df))
# df = df[columns]
logs+=f"Read {len(df)} rows files {csv_files} into df \n"

read C:\Users\eshad\OneDrive\Desktop\TU Delft\Thesis\Web Scrape PoC\Data\1. Consolidated data\2018\data_2018 backup 29 march\data_2018_1.csv
read C:\Users\eshad\OneDrive\Desktop\TU Delft\Thesis\Web Scrape PoC\Data\1. Consolidated data\2018\data_2018 backup 29 march\data_2018_10.csv
read C:\Users\eshad\OneDrive\Desktop\TU Delft\Thesis\Web Scrape PoC\Data\1. Consolidated data\2018\data_2018 backup 29 march\data_2018_2.csv
read C:\Users\eshad\OneDrive\Desktop\TU Delft\Thesis\Web Scrape PoC\Data\1. Consolidated data\2018\data_2018 backup 29 march\data_2018_3.csv
read C:\Users\eshad\OneDrive\Desktop\TU Delft\Thesis\Web Scrape PoC\Data\1. Consolidated data\2018\data_2018 backup 29 march\data_2018_4.csv
read C:\Users\eshad\OneDrive\Desktop\TU Delft\Thesis\Web Scrape PoC\Data\1. Consolidated data\2018\data_2018 backup 29 march\data_2018_5.csv
read C:\Users\eshad\OneDrive\Desktop\TU Delft\Thesis\Web Scrape PoC\Data\1. Consolidated data\2018\data_2018 backup 29 march\data_2018_6.csv
read C:\User

## Data Preprocessing & Language Filtering

In [70]:
def literal_eval_func(x):
    try:
        return ast.literal_eval(x.replace("\n", " ").replace("\'False\'", "\"FALSE\""))
    except Exception as e:
        return "literal eval failed"

In [71]:
# df['description'] = df['description'].apply(lambda x: ast.literal_eval(x))
df['description'] = df['description'].apply(lambda x: literal_eval_func(x))
df['description'] = df['description'].apply(lambda x: x['text'] if 'text' in x else None)
# df.head()

In [72]:
print(len(df))
df = df[df['description']!="literal eval failed"]
df = df[~df['description'].isna()]
print(len(df))

84084
84084


In [73]:
def extract_language(desc, title):
    # if not desc or not title:
        # return "not found"
    try:
        return detect(desc)
    except:
        try:
            return detect(title)
        except:
            # print(title, desc)
            return "not found"

DetectorFactory.seed = 0
df['language'] = df.apply(lambda x: extract_language(x['description'], x['title']), axis = 1)
print(len(df))

84084


In [74]:
print(len(df[df['language']=="not found"]))
df[df['language']=="not found"]
logs+=f"Language not found for {len(df[df['language']=="not found"])} rows \n"

0


In [75]:
df_eng = df[df['language']=="en"]
print(len(df_eng))
logs+=f"English language jobs: {len(df_eng)} rows \n"

75194


In [76]:
myCoder = SOCCoder()
df_eng['soc_code'] = df_eng.apply(lambda x: myCoder.code_record(str(x['title']), str(x['description'])), axis = 1)

In [77]:
def extract_localized_name(dict):
    try:
        if  'On-site' in dict:
            return 'On-site'
        elif 'Hybrid' in dict:
            return 'Hybrid'
        elif 'Remote' in dict:
            return 'Remote'
        return None
    except:
        return None

df_eng['work_model'] = df_eng['workplaceTypesResolutionResults'].apply(lambda x: extract_localized_name(x))
# df_eng.head()

In [78]:
df_eng = df_eng[['jobPostingId', 'companyDetails', 'title', 'description', 'workRemoteAllowed', 'formattedLocation', 'work_model', 'soc_code']] 
# print(len(df_eng))
df_eng.head()

,jobPostingId,companyDetails,title,description,workRemoteAllowed,formattedLocation,work_model,soc_code
0,589586880,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Regulatory Affairs Project Manager,SUMMARY OF POSITION: \n This position will be ...,False,"Mansfield, MA",None,242
1,589586718,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Senior Corporate Accountant,"Location: Blue Bell, PA \n Corporate Accountin...",False,"Philadelphia, PA",None,412
2,589586238,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Sr. Transportation Engineer,Our Company is going through rapid growth and ...,False,"Kirkland, WA",None,212
3,589586172,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Programs Sales Manager,The District of Columbia Bar has an immediate ...,False,"Arlington, VA",None,354
4,589585092,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Warranty Engineer,This position has direct responsibility to pla...,False,"Plymouth, MI",None,353


## Filtering SOC Code
- SOC code filtered: 213
- Keywords: ['devops', 'data science', 'data scientist', 'machine learning', 'site reliability', 'data analyst', 'data engineer', 'information technology', 'blockchain', 'cyber security', 'cybersecurity', 'database administrator', 'database admin', 'system administrator', 'system admin', 'systems admin', 'computer science', 'computer scientist', 'full stack', 'full-stack', 'back end', 'back-end', 'front end', 'front-end', 'mobile dev', 'game dev', 'graphics dev', 'cloud infrastructure', 'ai developer', 'qa dev', 'test dev']
- 
['IT ', 'AI ', 'AI/']

In [79]:
len(df_eng)

75194

In [80]:
keywords_to_retain = ['devops', 'data science', 'data scientist', 'machine learning', 'site reliability', 'data analyst', 'data engineer', 'information technology', 'blockchain', 'cyber security', 'cybersecurity', 'database administrator', 'database admin', 'system administrator', 'system admin', 'systems admin', 'computer science', 'computer scientist', 'full stack', 'full-stack', 'back end', 'back-end', 'front end', 'front-end', 'mobile dev', 'game dev', 'graphics dev', 'cloud infrastructure', 'ai developer', 'qa dev', 'test dev']
case_sensitive_keywords_to_retain = ['IT ', 'AI ', 'AI/']

In [81]:
pattern_keywords_to_retain = '|'.join(keywords_to_retain)
pattern_case_sensitive_keywords_to_retain = '|'.join(case_sensitive_keywords_to_retain)
print(pattern_keywords_to_retain)
print(pattern_case_sensitive_keywords_to_retain)

devops|data science|data scientist|machine learning|site reliability|data analyst|data engineer|information technology|blockchain|cyber security|cybersecurity|database administrator|database admin|system administrator|system admin|systems admin|computer science|computer scientist|full stack|full-stack|back end|back-end|front end|front-end|mobile dev|game dev|graphics dev|cloud infrastructure|ai developer|qa dev|test dev
IT |AI |AI/


In [82]:
cs_jobs_keywords_1 = df_eng[df_eng['title'].str.contains(pattern_keywords_to_retain, case=False, na=False)]
cs_jobs_keywords_2 = df_eng[df_eng['title'].str.contains(pattern_case_sensitive_keywords_to_retain, case=True, na=False)]
print(len(cs_jobs_keywords_1), len(cs_jobs_keywords_2))

2396 921


In [83]:
cs_jobs_code_213 = df_eng[df_eng['soc_code']=='213']
print(len(cs_jobs_code_213))
# cs_jobs_code_213.head()

12605


In [84]:
cs_jobs = pd.concat([cs_jobs_keywords_1, cs_jobs_keywords_2, cs_jobs_code_213])
print(len(cs_jobs))
# cs_jobs.head()

15922


In [85]:
logs+=f"CS jobs found = {len(cs_jobs)} rows \n"
print(len(cs_jobs))

15922


In [86]:
cs_jobs.to_csv(f"{save_path}")